In [1]:
import nest_asyncio
nest_asyncio.apply()

import os, json, time
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters


In [ ]:
# Configuration
BOT_TOKEN = "7556063014:AAGmI-zrPbqorpGaQAVt7yOvXiems3ciaY0"
JOB_DIR = "../jobs"

# Ensure jobs directory exists
os.makedirs(JOB_DIR, exist_ok=True)

# Temporary buffer for awaiting profile links
user_pending_links = {}

# Command reference text
COMMAND_REFERENCE = """🤖 MegaSeleniumBot Command Reference

1. Post a Tweet
/tweet <profile_name> <tweet_context>
Example: /tweet ubhay AI is changing the nature of education.

2. Post a LinkedIn Update 
/linkedin <profile_name> <post_context>
Example: /linkedin ayush Founders and Heads of Product in early-stage startups

3. Auto-Send LinkedIn Connect Messages
Step 1: /connect <profile_name> <persona>
Step 2: Send comma-separated LinkedIn profile URLs
Example: 
/connect ayush Let's connect around community-led product growth.
Then send: https://linkedin.com/in/a, https://linkedin.com/in/b

4. Tag Twitter Accounts in a Tweet
Step 1: /tag <profile_name> <tweet_context>
Step 2: Send comma-separated Twitter profile URLs
Example: 
/tag ubhay Shoutout to these builders in AI infra
Then send: https://twitter.com/a, https://twitter.com/b, https://twitter.com/c

5. Auto-Comment on Latest Tweets
Step 1: /comment_twitter <profile_name> <comment_context>
Step 2: Send comma-separated Twitter profile URLs
Example: 
/comment_twitter ubhay Highlighting strong hiring frameworks.
Then send: https://twitter.com/user1, https://twitter.com/user2

Notes:
• All commands must begin with /
• profile_name must match an existing folder inside /profiles/
• For commands 3, 4, and 5: First send the command, then send profile links in a follow-up message
• All content is generated using ChatGPT for personalization
• Profile links are saved in JSON for automation scripts to process"""

async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle /start command - show command reference"""
    await update.message.reply_text(
        f"Welcome to MegaSeleniumBot! 🚀\n\n{COMMAND_REFERENCE}"
    )

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle /help command - show command reference"""
    await update.message.reply_text(COMMAND_REFERENCE)

async def handle_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle all automation commands"""
    text = update.message.text.strip()
    sender = update.effective_user.username or str(update.effective_user.id)
    timestamp = int(time.time())
    
    parts = text.split(" ", 2)
    
    # Validate command format
    if len(parts) < 3:
        await update.message.reply_text(
            "❌ Invalid format!\n\n"
            "Correct format: /<command> <profile_name> <content>\n\n"
            "Use /help to see all available commands."
        )
        return
    
    cmd_type = parts[0][1:].lower()  # Remove '/' and convert to lowercase
    profile_user = parts[1]
    content = parts[2]
    
    # Validate command type
    valid_commands = ["tweet", "linkedin", "connect", "tag", "comment_twitter"]
    if cmd_type not in valid_commands:
        await update.message.reply_text(
            f"❌ Unknown command: {cmd_type}\n\n"
            f"Valid commands: {', '.join(valid_commands)}\n\n"
            "Use /help to see all available commands."
        )
        return
    
    # Create job object
    job = {
        "type": cmd_type,
        "user": profile_user,
        "content": content,
        "sender": sender,
        "timestamp": timestamp
    }
    
    # Commands that need profile links in follow-up message
    if cmd_type in ["connect", "tag", "comment_twitter"]:
        user_pending_links[sender] = {
            "job": job,
            "expecting_links": True,
            "command_type": cmd_type
        }
        
        if cmd_type == "connect":
            await update.message.reply_text(
                f"📋 Step 2 for LinkedIn Connect:\n\n"
                f"Send comma-separated LinkedIn profile URLs:\n"
                f"https://linkedin.com/in/profile1, https://linkedin.com/in/profile2, ...\n\n"
                f"⏳ Waiting for your LinkedIn profile links..."
            )
        elif cmd_type == "tag":
            await update.message.reply_text(
                f"📋 Step 2 for Twitter Tag:\n\n"
                f"Send comma-separated Twitter profile URLs:\n"
                f"https://twitter.com/profile1, https://twitter.com/profile2, ...\n\n"
                f"⏳ Waiting for your Twitter profile links..."
            )
        elif cmd_type == "comment_twitter":
            await update.message.reply_text(
                f"📋 Step 2 for Twitter Comment:\n\n"
                f"Send comma-separated Twitter profile URLs:\n"
                f"https://twitter.com/user1, https://twitter.com/user2, ...\n\n"
                f"⏳ Waiting for your Twitter profile links..."
            )
    else:
        # Commands that don't need follow-up (tweet, linkedin)
        write_job(job)
        await update.message.reply_text(
            f"✅ Job recorded successfully!\n\n"
            f"**Type:** {cmd_type}\n"
            f"**Profile:** {profile_user}\n"
            f"**Content:** {content[:50]}{'...' if len(content) > 50 else ''}\n\n"
            f"Job will be processed by the automation system.",
            parse_mode='Markdown'
        )

async def handle_followup_links(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Handle follow-up messages with profile links"""
    sender = update.effective_user.username or str(update.effective_user.id)
    text = update.message.text.strip()
    
    # Check if user has pending links
    if sender not in user_pending_links:
        await update.message.reply_text(
            "❌ No pending command found.\n\n"
            "Please use a command first (e.g., /connect, /tag, /comment_twitter) "
            "before sending profile links.\n\n"
            "Use /help to see all available commands."
        )
        return
    
    # Parse and validate links
    raw_links = [link.strip() for link in text.split(",") if link.strip()]
    
    if not raw_links:
        await update.message.reply_text(
            "❌ No valid links found!\n\n"
            "Please send comma-separated profile URLs:\n"
            "`https://linkedin.com/in/profile1, https://twitter.com/profile2, ...`",
            parse_mode='Markdown'
        )
        return
    
    # Validate link format based on command type
    job_data = user_pending_links[sender]["job"]
    command_type = user_pending_links[sender]["command_type"]
    
    invalid_links = []
    valid_links = []
    
    for link in raw_links:
        if command_type == "connect":
            if "linkedin.com" in link:
                valid_links.append(link)
            else:
                invalid_links.append(link)
        elif command_type in ["tag", "comment_twitter"]:
            if "twitter.com" in link or "x.com" in link:
                valid_links.append(link)
            else:
                invalid_links.append(link)
        else:
            valid_links.append(link)  # For other commands, accept all links
    
    if invalid_links:
        expected_domain = "LinkedIn" if command_type == "connect" else "Twitter/X"
        await update.message.reply_text(
            f"⚠️ Some links are invalid for {command_type} command:\n\n"
            f"**Invalid links:** {', '.join(invalid_links)}\n\n"
            f"Expected {expected_domain} profile links. Please resend with correct URLs.",
            parse_mode='Markdown'
        )
        return
    
    # Add links to job and save
    job_data["profile_links"] = valid_links
    write_job(job_data)
    
    # Send confirmation with profile links preview
    links_preview = []
    for i, link in enumerate(valid_links[:3]):  # Show first 3 links
        links_preview.append(f"• {link}")
    
    links_text = "\n".join(links_preview)
    if len(valid_links) > 3:
        links_text += f"\n• ... and {len(valid_links) - 3} more"
    
    await update.message.reply_text(
        f"✅ **Job + Profile Links Recorded Successfully!**\n\n"
        f"**Type:** {job_data['type']}\n"
        f"**Profile:** {job_data['user']}\n"
        f"**Content:** {job_data['content'][:50]}{'...' if len(job_data['content']) > 50 else ''}\n"
        f"**Target Profiles ({len(valid_links)}):**\n{links_text}\n\n"
        f"✨ Job will be processed by the automation system.\n"
        f"📄 Saved as: `{job_data['type']}_{job_data['user']}_{job_data['timestamp']}.json`",
        parse_mode='Markdown'
    )
    
    # Clean up pending links
    del user_pending_links[sender]

def write_job(job):
    """Write job to JSON file"""
    try:
        filename = f"{job['type']}_{job['user']}_{job['timestamp']}.json"
        filepath = os.path.join(JOB_DIR, filename)
        
        with open(filepath, "w", encoding="utf-8") as f:
            json.dump(job, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Job written: {filename}")
        
    except Exception as e:
        print(f"❌ Error writing job: {e}")

async def status_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Show bot status and pending jobs"""
    sender = update.effective_user.username or str(update.effective_user.id)
    
    status_text = "📊 **Bot Status**\n\n"
    
    # Check pending links
    if sender in user_pending_links:
        pending = user_pending_links[sender]
        status_text += f"⏳ **Pending Action:** Waiting for profile links\n"
        status_text += f"**Command:** {pending['command_type']}\n"
        status_text += f"**Profile:** {pending['job']['user']}\n\n"
    else:
        status_text += "✅ No pending actions\n\n"
    
    # Check jobs directory
    try:
        job_files = [f for f in os.listdir(JOB_DIR) if f.endswith('.json')]
        status_text += f"📁 **Jobs Directory:** {len(job_files)} job file(s)\n"
    except:
        status_text += "📁 **Jobs Directory:** Unable to read\n"
    
    status_text += "\nUse /help to see all available commands."
    
    await update.message.reply_text(status_text, parse_mode='Markdown')

async def cancel_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
    """Cancel pending operations"""
    sender = update.effective_user.username or str(update.effective_user.id)
    
    if sender in user_pending_links:
        del user_pending_links[sender]
        await update.message.reply_text(
            "❌ **Pending operation cancelled**\n\n"
            "You can start a new command anytime.\n"
            "Use /help to see all available commands.",
            parse_mode='Markdown'
        )
    else:
        await update.message.reply_text(
            "ℹ️ No pending operations to cancel.\n\n"
            "Use /help to see all available commands."
        )

def main():
    """Main function to run the bot"""
    print("🚀 Starting MegaSeleniumBot...")
    
    # Create application
    app = ApplicationBuilder().token(BOT_TOKEN).build()
    
    # Add command handlers
    app.add_handler(CommandHandler("start", start_command))
    app.add_handler(CommandHandler("help", help_command))
    app.add_handler(CommandHandler("status", status_command))
    app.add_handler(CommandHandler("cancel", cancel_command))
    
    # Add handlers for automation commands
    command_list = ["tweet", "linkedin", "connect", "tag", "comment_twitter"]
    app.add_handler(CommandHandler(command_list, handle_command))
    
    # Add handler for follow-up messages (profile links)
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_followup_links))
    
    print("✅ Bot is ready and polling for messages...")
    print(f"📁 Jobs will be saved to: {os.path.abspath(JOB_DIR)}")
    
    # Start polling
    app.run_polling()

if __name__ == "__main__":
    main()

🚀 Starting MegaSeleniumBot...
✅ Bot is ready and polling for messages...
📁 Jobs will be saved to: c:\Users\abhay\Desktop\bots\jobs


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\ext\_application.py", line 1309, in process_update
    await coroutine
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\ext\_handlers\basehandler.py", line 158, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\abhay\AppData\Local\Temp\ipykernel_17320\3208964468.py", line 62, in handle_command
    text = update.message.text.strip()
           ^^^^^^^^^^^^^^^^^^^
AttributeError: 'NoneType' object has no attribute 'text'


✅ Job written: tweet_ubhay_1752666215.json
✅ Job written: tweet_ubhay_1752666500.json
✅ Job written: tweet_ubhay_1752666743.json
✅ Job written: tweet_ubhay_1752666832.json
✅ Job written: tweet_ubhay_1752666843.json
✅ Job written: linkedin_ubhay_1752666888.json


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpx\_transports\default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_async\connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_async\connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\

✅ Job written: linkedin_ubhay_1752672327.json
✅ Job written: linkedin_ubhay_1752672951.json
✅ Job written: linkedin_ubhay_1752673031.json
✅ Job written: linkedin_ubhay_1752673043.json
✅ Job written: linkedin_ubhay_1752673179.json
✅ Job written: linkedin_ubhay_1752673396.json
✅ Job written: linkedin_ubhay_1752676879.json
✅ Job written: linkedin_ubhay_1752676949.json


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpx\_transports\default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_async\connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_async\connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\

✅ Job written: linkedin_ubhay_1752740043.json
✅ Job written: linkedin_ubhay_1752740060.json


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpx\_transports\default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_async\connection_pool.py", line 256, in handle_async_request
    raise exc from None
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\Lib\site-packages\httpcore\_async\connection_pool.py", line 236, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\abhay\AppData\Local\Programs\Python\Python312\